In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote_plus, urlencode, urlparse, urlunparse, unquote
import pandas as pd
import re
import json
from collections import Counter
import pandas as pd

In [3]:
data = pd.read_pickle("/data/rali7/Tmp/solimanz/data/pickles/clean_2017_11_28.pkl")

In [4]:
func = data.transformed.value_counts()

In [5]:
del data

In [5]:
import pickle
with open("/data/rali7/Tmp/solimanz/data/pickles/wiki_urls_v2.pkl", "rb") as f:
    urls = pickle.load(f)

# Fetch Relevant Wikipedia URLs from Bing

In [6]:
urls = set()
not_crawled = []

for title in func.index.values[5000:10000]:
    payload = {"q":f"site:en.wikipedia.org {title}"}
    result = urlencode(payload, quote_via=quote_plus)
    response = urlopen(f"http://www.bing.com/search?{result}")
    if response.getcode() == 200:
        bs = BeautifulSoup(response.read(), "html5lib")
        anchors = bs.find_all("a")
        anchors = {urlparse(a["href"]) for a in anchors 
                   if a.has_attr("href") and urlparse(a["href"]).netloc == "en.wikipedia.org"}
        urls = urls.union({url.netloc + url.path for url in anchors})
    else:
        not_crawled.append(f"http://www.bing.com/search?{result}")

In [136]:
urls_bak = set(urls)

In [6]:
len(urls)

39511

In [12]:
urls = urls_bak.union(urls)

In [13]:
import pickle
with open("/data/rali7/Tmp/solimanz/data/pickles/wiki_urls_v2.pkl", "wb") as f:
    pickle.dump(urls, f)

# Scraping Wikipedia

In [7]:
import wikipedia

In [9]:
pages = []
for url in urls:
    try:
        complete = "http://" + url
        page = urlparse(complete).path.split('/')[-1]
        page = unquote(page).replace("_", " ")
        pages.append(wikipedia.page(page).content)
    
    except wikipedia.DisambiguationError:
        continue
    except wikipedia.PageError:
        continue

/u/solimanz/development/career_prediction_gpu/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /u/solimanz/.pyenv/versions/3.6.3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


SSLError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=Student&format=json&action=query (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:777)'),))

In [15]:
with open("/data/rali7/Tmp/solimanz/data/pickles/wiki_pages.pkl", "wb") as f:
    pickle.dump(pages, f)

In [17]:
with open("/data/rali7/Tmp/solimanz/data/wikipedia/pages.txt", 'w') as f:
    for page in pages:
        f.write(page)
        f.write("\n")

In [1]:
with open("/data/rali7/Tmp/solimanz/data/wikipedia/pages.txt", 'r') as f:
    text = f.read()

In [3]:
text = text.lower()

'frances wright (september 6, 1795 – december 13, 1852) also widely known as fanny wright, was a scottish-born lecturer, writer, freethinker, feminist, abolitionist, and social reformer, who became a us citizen in 1825. the same year, she founded the nashoba commune in tennessee, as a utopian community to prepare slaves for emancipation, but it lasted only five years. her views of society and manners in america (1821) brought her to public attention as a critic of the new nation.\n\n\n== early life and education ==\nshe was one of three children born in dundee, scotland, to camilla campbell and james wright, a wealthy linen manufacturer and political radical. her father designed dundee trade tokens, knew adam smith and corresponded with french republicans, including gilbert du motier, marquis de lafayette. both parents died young, but fanny (as she was called as a child), orphaned at the age of three, was left a substantial inheritance. her maternal aunt became her guardian and took fa